In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.2.2%2Bcu121-cp311-cp311-win_amd64.whl (2454.8 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.17.2%2Bcu121-cp311-cp311-win_amd64.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.2.2%2Bcu121-cp311-cp311-win_amd64.whl (4.1 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.9.0-py3-none-any.whl (9.7 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached https://download.pytorch.org/whl/fsspec-2023.4.0-py3-none-any.whl (153 kB)
     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     - -------------------------------------- 0.6/15.8 MB 11.5 MB/s eta 0:00:02
     --- ------------------------------------ 1.5/15.8 MB 15.6 MB/s eta 0:00:01
     ----- ---

In [3]:
pip install torch_geometric

     ---------------------------------------- 0.0/64.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/64.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/64.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/64.2 kB 162.5 kB/s eta 0:00:01
     ------------------ ------------------- 30.7/64.2 kB 163.8 kB/s eta 0:00:01
     ------------------------ ------------- 41.0/64.2 kB 178.6 kB/s eta 0:00:01
     ------------------------------ ------- 51.2/64.2 kB 201.8 kB/s eta 0:00:01
     -------------------------------------- 64.2/64.2 kB 229.4 kB/s eta 0:00:00
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     -------------------------------------- 60.6/60.6 kB 799.0 kB/s eta 0:00:00
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ----------------------------

In [4]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas

  Using cached pandas-2.2.1-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [9]:
#Check everything is connected properly
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name())
print("PyTorch version:", torch.__version__)
!nvidia-smi

True
0
NVIDIA GeForce RTX 3070
PyTorch version: 2.2.2+cu121
Thu Apr  4 06:59:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.61                 Driver Version: 551.61         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070      WDDM  |   00000000:01:00.0  On |                  N/A |
| 35%   29C    P8             18W /  220W |     758MiB /   8192MiB |     28%      Default |
|                                         |                        |            

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split

def read_edge_list(filepath) :
  with open(filepath, 'r') as file:
    edge_list_content = file.read()
    return edge_list_content

# Define your Graph Neural Network model
class GCNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNClassifier, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Assuming you have your edge lists and labels
# Load your data into PyTorch Geometric format
# Construct your graphs and labels
# Example:
# edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
# x = torch.tensor([[0, 1], [1, 0], [0, 1]], dtype=torch.float)  # Node features

edge_list_content1 = read_edge_list("data/test1.txt")
edge_list_content2 = read_edge_list("data/test2.txt")
edge_list_content3 = read_edge_list("data/test3.txt")

# Define your dataset
class GraphDataset(torch.utils.data.Dataset):
    def __init__(self, graphs, labels):
        self.graphs = graphs
        self.labels = labels

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx], self.labels[idx]

# Split your data into train/validation/test sets
train_graphs, test_graphs, train_labels, test_labels = train_test_split(graphs, labels, test_size=0.2)
train_graphs, val_graphs, train_labels, val_labels = train_test_split(train_graphs, train_labels, test_size=0.2)

# Create DataLoader for each set
train_dataset = GraphDataset(train_graphs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = GraphDataset(val_graphs, val_labels)
val_loader = DataLoader(val_dataset, batch_size=64)
test_dataset = GraphDataset(test_graphs, test_labels)
test_loader = DataLoader(test_dataset, batch_size=64)

# Initialize your model, loss function, and optimizer
model = GCNClassifier(input_dim, hidden_dim, output_dim)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
def train(model, loader, optimizer, criterion):
    model.train()
    for data, labels in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

# Evaluation loop
def evaluate(model, loader, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in loader:
            out = model(data)
            _, predicted = torch.max(out, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Train and validate the model
for epoch in range(num_epochs):
    train(model, train_loader, optimizer, criterion)
    val_accuracy = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}, Validation Accuracy: {val_accuracy}")

# Test the final model
test_accuracy = evaluate(model, test_loader, criterion)
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
def read_edge_list(filepath) :
      with open(filepath, 'r') as file:
    edge_list_content = file.read()
    return edge_list_content

edge_list_content1 = read_edge_list("data/testing/test1.txt")
edge_list_content2 = read_edge_list("data/testing/test2.txt")
edge_list_content3 = read_edge_list("data/testing/test3.txt")

print(edge_list_content1)
print(edge_list_content2)
print(edge_list_content3)